In [ ]:
import requests
import json
import os
import pandas as pd
import numpy as np

from loguru import logger

In [ ]:
def calculate_distance_matrix(locations):
    url = "http://192.168.62.107:8002/route"

    # Prepare the JSON payload for Valhalla
    payload = {
        "locations": locations,
        "costing": "auto",
        "directions_options": {"units": "kilometers"},
        "matrix_type": "one_to_many",
    }

    # Make the HTTP POST request to Valhalla
    response = requests.post(url, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        try:
            distance = response.json()['trip']['summary']['length']
            return distance
        except KeyError:
            print("Error: 'distance' key not found in the response.")
            return None
    else:

        print(f"Error {response.status_code}: {response.text}")
        return None

In [ ]:
cust_loc = np.zeros(shape=(300, 2), dtype=np.float32)

for i, item in enumerate(ge_list[0: 300]):
    cust_loc[i, 0] = item['lat']
    cust_loc[i, 1] = item['lon']

np.save('../data/loc', cust_loc)

In [ ]:
n = len(ge_list[0: 300])
distance_matrix = np.zeros((n, n), dtype=np.float32)
batchsize = 100

for i in tqdm(range(0, n, batchsize)):
    for j in range(0, n, batchsize):
        matrix = np.zeros(shape=(batchsize, batchsize), dtype=np.float32)
        location_i = ge_list[i: min(i + batchsize, n)]
        location_j = ge_list[j: min(j + batchsize, n)]
        for idx_i, loc_i in enumerate(location_i):
            for idx_j, loc_j in enumerate(location_j):
                matrix[idx_i, idx_j] = calculate_distance_matrix([loc_i, loc_j])
        distance_matrix[i: min(i + batchsize, n), j: min(j + batchsize, n)] = matrix